In [16]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

import pandas as pd
import numpy as np


# 数据集路径
train_data_path = '/Users/yuchk/PycharmProjects/IMDB/0_dataset/orign/train_imdb.tsv'
test_data_path = '/Users/yuchk/PycharmProjects/IMDB/0_dataset/orign/test_imdb.tsv'

df_train = pd.read_csv(train_data_path,usecols=['tag', 'sen'], sep='\t')
df_test = pd.read_csv(test_data_path,usecols=['tag', 'sen'], sep='\t')

sen_list = [sen for sen in df_train["sen"]]
sen_list.extend([sen for sen in df_test["sen"]])

tag_list = [sen for sen in df_train["tag"]]
tag_list.extend([sen for sen in df_test["tag"]])

In [32]:
tokenizer = Tokenizer()
seq=[]
seqtence=[]
stop_words=set(stopwords.words('english'))
for con in sen_list:
    words=nltk.word_tokenize(con)
    line=[]
    for word in words:
        if word.isalpha() and word not in stop_words:
            line.append(word)
    seq.append(line)
    tokenizer.fit_on_texts(line)
    seqtence.extend(line)
# 获取词索引


word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(seq)
# 此处设置每个句子最长不超过 800


In [34]:
# 转换为numpy类型
label=np.array(tag_list)


# 此处设置每个句子最长不超过 800
final_sequences=sequence.pad_sequences(sequences,maxlen=800)

X=final_sequences
y=label

In [35]:
print(X[5],y)

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [24]:
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2)

In [7]:
# 网络构建
model=Sequential()
model.add(Embedding(89483,256,input_length=800))
model.add(LSTM(128,dropout=0.2))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(Xtrain,ytrain,batch_size=32,epochs=10,validation_data=(Xtest,ytest))


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 828s 41ms/step - loss: 0.6937 - accuracy: 0.5023 - val_loss: 0.6934 - val_accuracy: 0.5002
Epoch 2/10
20000/20000 [==============================] - 880s 44ms/step - loss: 0.6934 - accuracy: 0.5014 - val_loss: 0.6932 - val_accuracy: 0.4998
Epoch 3/10
20000/20000 [==============================] - 903s 45ms/step - loss: 0.6932 - accuracy: 0.5053 - val_loss: 0.6933 - val_accuracy: 0.5002
Epoch 4/10
20000/20000 [==============================] - 1034s 52ms/step - loss: 0.6933 - accuracy: 0.4973 - val_loss: 0.6932 - val_accuracy: 0.5002
Epoch 5/10
20000/20000 [==============================] - 863s 43ms/step - loss: 0.6932 - accuracy: 0.4966 - val_loss: 0.6931 - val_accuracy: 0.5002
Epoch 6/10
20000/20000 [==============================] - 911s 46ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.4998
Epoch 7/10
20000/20000 [==============================]

In [8]:
y_pre = model.predict(Xtest)

In [12]:
y_pre

array([[0.4978517],
       [0.4978517],
       [0.4978517],
       ...,
       [0.4978517],
       [0.4978517],
       [0.4978517]], dtype=float32)

In [15]:
print(Xtrain,Xtest,ytrain,ytest)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] [0 1 0 ... 0 0 0] [0 1 1 ... 0 0 1]


In [10]:
from sklearn.metrics import accuracy_score

In [11]:
sgdr = accuracy_score(ytest, y_pre)
print(sgdr)

ValueError: Classification metrics can't handle a mix of binary and continuous targets